In [1]:
import time, os, pdb, pickle, argparse, shutil, yaml, torch, math, time, pdb, datetime, pickle, sys
import utils #file
from solver_encoder import Solver 
from data_loader import pathSpecDataset
from torch.utils.data import DataLoader
from torch.backends import cudnn
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import medfilt
import torch.nn.functional as F
import importlib

In [2]:
# tailor config, define other
save_as = 'egs_metadata.pkl'
model_name = 'TrueVtesNoCd16Freq16Neck' #just for using models config file and vte_model
cudnn.benchmark = True
use_avg_vte = False
convert_style = True #if you plan to just resynthesize the examples or actually convert the styles
use_meta_list = True
ckpt_iters = 310000
autovc_model_saves_dir = '/homes/bdoc3/my_data/autovc_data/vte-autovc/model_saves/'
autovc_model_dir = autovc_model_saves_dir + model_name
config = pickle.load(open(autovc_model_dir +'/config.pkl','rb'))
config.batch_size = 1
config.autovc_ckpt = autovc_model_dir +'/ckpts/ckpt_' +str(ckpt_iters) +'.pth.tar'
avg_embs = np.load(os.path.dirname(config.emb_ckpt) +'/averaged_embs.npy')
config.vte_ckpt = '/homes/bdoc3/phonDet/results/newStandardAutovcSpmelParamsUnnormLatent64Out256/best_epoch_checkpoint.pth.tar'
# additional config attrs
style_names = ['belt','lip_trill','straight','vocal_fry','vibrato','breathy']
singer_names = ['m1_','m2_','m3_','m4_','m5_','m6_','m7_','m8_','m9_','m10_','m11_','f1_','f2_','f3_','f4_','f5_','f6_','f7_','f8_','f9_']
test_names = pickle.load(open(os.path.dirname(config.emb_ckpt) +'/config_params.pkl', 'rb')).test_list.split(' ')
train_names = [item for item in singer_names if item not in test_names]

config.exclude_list = train_names

male_idx = range(0,11)
female_idx = range(11,20)
config.device = torch.device(f'cuda:{config.which_cuda}' if torch.cuda.is_available() else 'cpu')
with open(config.spmel_dir +'/spmel_params.yaml') as File:
    spmel_params = yaml.load(File, Loader=yaml.FullLoader)
config

Namespace(adam_init=0.0001, autovc_ckpt='/homes/bdoc3/my_data/autovc_data/vte-autovc/model_saves/TrueVtesNoCd16Freq16Neck/ckpts/ckpt_310000.pth.tar', batch_size=1, chunk_num=6, chunk_seconds=0.5, ckpt_freq=10000, ckpt_model='', data_dir='/homes/bdoc3/my_data/autovc_data/vte-autovc/model_saves', device=device(type='cuda', index=0), dim_emb=256, dim_neck=16, dim_pre=512, emb_ckpt='/homes/bdoc3/phonDet/results/newStandardAutovcSpmelParamsUnnormLatent64Out256/best_epoch_checkpoint.pth.tar', exclude_list=['m1_', 'm4_', 'm5_', 'm6_', 'm7_', 'm8_', 'm9_', 'm11_', 'f1_', 'f3_', 'f5_', 'f6_', 'f7_', 'f8_', 'f9_'], file_name='TrueVtesNoCd16Freq16Neck', freq=16, lambda_cd=1, len_crop=192, log_step=10, num_iters=500000, one_hot=False, prnt_loss_weight=1.0, psnt_loss_weight=1.0, spec_freq=10000, spmel_dir='/homes/bdoc3/my_data/phonDet/spmel_autovc_params_unnormalized', train_size=21, use_ckpt_config=False, vte_ckpt='/homes/bdoc3/phonDet/results/newStandardAutovcSpmelParamsUnnormLatent64Out256/best_

In [3]:
# setup dataloader, models
vocalSet = pathSpecDataset(config, spmel_params)
vocalSet_loader = DataLoader(vocalSet, batch_size=config.batch_size, shuffle=True, drop_last=False)
vte = utils.setup_vte(config, spmel_params)

f2_arpeggios_belt_a.npy
f2_arpeggios_belt_e.npy
f2_arpeggios_belt_i.npy
f2_arpeggios_belt_o.npy
f2_arpeggios_belt_u.npy
f2_arpeggios_breathy_a.npy
f2_arpeggios_breathy_e.npy
f2_arpeggios_breathy_i.npy
f2_arpeggios_breathy_o.npy
f2_arpeggios_breathy_u.npy
f2_arpeggios_lip_trill_a.npy
f2_arpeggios_lip_trill_e.npy
f2_arpeggios_lip_trill_i.npy
f2_arpeggios_lip_trill_o.npy
f2_arpeggios_lip_trill_u.npy
f2_arpeggios_straight_a.npy
f2_arpeggios_straight_e.npy
f2_arpeggios_straight_i.npy
f2_arpeggios_straight_o.npy
f2_arpeggios_straight_u.npy
f2_arpeggios_vibrato_a.npy
f2_arpeggios_vibrato_e.npy
f2_arpeggios_vibrato_i.npy
f2_arpeggios_vibrato_o.npy
f2_arpeggios_vibrato_u.npy
f2_arpeggios_vocal_fry_a.npy
f2_arpeggios_vocal_fry_e.npy
f2_arpeggios_vocal_fry_i.npy
f2_arpeggios_vocal_fry_o.npy
f2_arpeggios_vocal_fry_u.npy
f2_scales_belt_a.npy
f2_scales_belt_e.npy
f2_scales_belt_i.npy
f2_scales_belt_o.npy
f2_scales_belt_u.npy
f2_scales_breathy_a.npy
f2_scales_breathy_e.npy
f2_scales_breathy_i.npy
f2_

In [4]:
sys.path.insert(1, '/homes/bdoc3/my_data/autovc_data') # usually the cwd is priority, so index 1 is good enough for our purposes here
print(sys.path)
from hparams import hparams
import torch

['/homes/bdoc3/vte-autovc', '/homes/bdoc3/my_data/autovc_data', '', '/import/linux/python/3.7.7/lib/python3.7/site-packages', '/import/linux/python/3.7.7/lib/python37.zip', '/import/linux/python/3.7.7/lib/python3.7', '/import/linux/python/3.7.7/lib/python3.7/lib-dynload', '/homes/bdoc3/.local/lib/python3.7/site-packages', '/homes/bdoc3/.local/lib/python3.7/site-packages/IPython/extensions', '/homes/bdoc3/.ipython']


In [12]:
find_male = True
num_examples = 8
use_unseen = True
counter = 0
data_iter = iter(vocalSet_loader)
if use_meta_list == True:
    example_meta_list = pickle.load(open(save_as, 'rb'))
else:
    example_meta_list = []

# start the spmel_emb generation cycle
while counter < num_examples:
    x_real, org_style_idx, singer_idx = next(data_iter)
    x_real = x_real.to(config.device)
    original_spmel = x_real[0].cpu().detach().numpy()
    if find_male == True:
        gender_idx = male_idx
    else:
        gender_idx = female_idx
    # if this example is the gender we're looking for
    if singer_idx in gender_idx:
        find_male = not find_male
        counter += 1
        # get source embedding
        if use_avg_vte == True:
            emb_org = torch.tensor(avg_embs[org_style_idx]).to(config.device).unsqueeze(0)
        else:
            x_real_chunked = x_real.view(x_real.shape[0]*config.chunk_num, x_real.shape[1]//config.chunk_num, -1)
            pred_style_idx, all_tensors = vte(x_real_chunked)
            emb_org = all_tensors[-1]
        example_meta_list.append((original_spmel, org_style_idx, singer_idx, emb_org))

In [13]:
len(example_meta_list)

16

In [9]:
with open(save_as,'wb') as handle:
    pickle.dump(example_meta_list,handle) 